# Software Developer Technical Interview Questions


* Author: Slaviana Pavlovich
* Date: 31/08/2021
* Programming Language: Python 3

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import unittest

# Creating the table that has tracks users’ video playback actions over time 
records = pd.DataFrame(data=np.matrix.transpose(np.array([[1,2,1,1,1,2,3], 
                                      ['Windows 10', 'OSX 15.4', 'Iphone 8s', 'Windows 10', 'Iphone 8s', 'OSX 15.4', 'Android 9.1'], 
                                      ['start','start','start','stop','stop','stop','start'], 
                                      [100,200,250,370,410,490,700]])),
                                      columns=['user_id', 'device', 'action','date_actioned'])    

# Converting user_id and date_actioned into integers
records[['date_actioned', 'user_id']] = records[['date_actioned', 'user_id']].apply(pd.to_numeric)

# Printing the records
print(records.to_string(index=False))


 user_id       device action  date_actioned
       1   Windows 10  start            100
       2     OSX 15.4  start            200
       1    Iphone 8s  start            250
       1   Windows 10   stop            370
       1    Iphone 8s   stop            410
       2     OSX 15.4   stop            490
       3  Android 9.1  start            700


## Task 1

Write a function that takes in records (an array of all the database records), an action, and a start_time and end_time time window and returns all user ids that performed that action within that time window. 

E.g. getUsers(records, “start”, 700, 900) will return the result [3]

In [2]:
''' 
The function that takes in records, action, start time and end time and returns the user(s) ids 
that performed that action within the time window given.
'''

def getUsers(df, action, start_time, end_time):
    # Identifying the users that match the criteria
    users_selected = df.loc[(df['action'].str.match(action)) & 
                               (df['date_actioned'] >= start_time) &
                               (df['date_actioned'] <= end_time), ['user_id']]
    
    return list(set(users_selected['user_id'].tolist()))


In [3]:
# Applying the function to the example scenario

action = 'start'
start_time = 700
end_time = 900

users = getUsers(records, action, start_time, end_time)
print(users)


[3]


## Task 2
 
Write a function that takes in a user_id and an array of all the database records, and reports a user’s total “unique” playback time in seconds.

E.g. getPlaybackTime(1, records) will return 310
 

In [4]:
''' 
The function that takes in user_id and the records, and the function 
output a user's total unique playback time in seconds
'''

def getPlaybackTime(user_id_selected, df):
    # Selecting the records of a particular user
    records_selected_df = df.loc[(df['user_id'] == user_id_selected), ['device','action','date_actioned']]
    user_devices = records_selected_df['device'].unique()
    device_times_array = []
    for device in user_devices:
        device_df = records_selected_df.loc[(df['device'] == device), ['device','action','date_actioned']]
        min = device_df['date_actioned'].min()
        max = device_df['date_actioned'].max()
        if min != max: #Exclude when only one start or stop
            device_times_array.append([min, max])
    sorted_array = sorted(device_times_array, key=lambda x: x[0])
    start = None
    end = None
    unique_playback_time = 0
    for s, e in sorted_array:
        
        if not start or not end:
            start = s
            end = e
        elif s <= end:
            start = end
            if e <= end:
                end = end #produces a null in the difference calculation
            else:
                end = e
        else:
            start = s
            end = e
        unique_playback_time += end - start
        
    return unique_playback_time


In [5]:
# Applying the function to the example scenario

uid = 1

playback_value = getPlaybackTime(uid, records)
print('User #', uid, 'has a total of ', playback_value,'(s) ', 'unique playback time.')


User # 1 has a total of  310 (s)  unique playback time.


## Task 3

Write inline comments for the functions where appropriate.

In [6]:
# Completed for Tasks 1, 2, 6

## Task 4

Write appropriate unit tests for your functions above

In [7]:
class Test_getUsers(unittest.TestCase):
    
    # Testing the outcome of the function for the given parameters
    def test_getUsers(self):
        self.assertEqual(getUsers(records, 'start', 700, 900), [3])  

unittest.main(argv=[''], verbosity=2, exit=False)

test_getUsers (__main__.Test_getUsers) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [8]:
class Test_getPlaybackTime(unittest.TestCase):
    
    # Testing the outcome of the function for the given parameters
    def test_getPlaybackTime(self):
        self.assertEqual(getPlaybackTime(1, records), 310)  

unittest.main(argv=[''], verbosity=2, exit=False)

test_getPlaybackTime (__main__.Test_getPlaybackTime) ... ok
test_getUsers (__main__.Test_getUsers) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.009s

OK


## Task 5

Identify any possible shortcomings or limitations of both your functions if any.

#### Task's 1 Limitations
1. No checks on the input values.

#### Task's 2 Limitations
1. The table contains records for one video.
2. We only check for one user at a time.
3. Each device is only started and stopped once at that period.
4. Devices that are only started or stopped are excluded from the calculation. 
5. No check if such user_id exists.

## Task 6

In [9]:
# A user has access to a set of applications:
user_apps = [{"app_id": 1}, 
             {"app_id": 2}, 
             {"app_id": 3}, 
             {"app_id": 126}]

# Each application has a series of features available:
app_features = [{"app_id": 1, "features_available": [1, 2, 3]}, 
                {"app_id": 2, "features_available": [3, 4, 5, 7]}, 
                {"app_id": 3, "features_available": [3, 12]}]

# Each user has feature specific access:
user_features = [{"user_id": 1, "features_allowed": [1, 2, 5]}, 
                 {"user_id": 2,"features_allowed": [1, 2, 3, 4]}, 
                 {"user_id": 3, "features_allowed": [5]}]

print(app_features)
print(user_features)


[{'app_id': 1, 'features_available': [1, 2, 3]}, {'app_id': 2, 'features_available': [3, 4, 5, 7]}, {'app_id': 3, 'features_available': [3, 12]}]
[{'user_id': 1, 'features_allowed': [1, 2, 5]}, {'user_id': 2, 'features_allowed': [1, 2, 3, 4]}, {'user_id': 3, 'features_allowed': [5]}]


Write a Python 3 function that takes in a user_id, the above three lists and returns a
dictionary object that looks something like this:

{
"user_id": 1,
"application_permissions": [
{
"app_id": 1,
"features_allowed": [1, 2]
},
{
"app_id": 2,
"features_allowed": [5]
},
{
"app_id": 3
"features_allowed": []
},
...
]
}

In other words, return a dictionary object which shows, for all allowed applications, the allowed
features for a user.

In [10]:
''' 
The function is used to get the target arrays in two cases: when extracting the features allowed 
for the user of interest and getting the features available for an app
'''

def get_target_array(lookup_value, lookup_key, output_key, input_array):
    for line in input_array:
        if line[lookup_key] == lookup_value:
            return line[output_key]
    return []
 
    
''' 
The function that takes in user_id and and the three lists and returns a dictionary object 
which shows for all allowed applications, the allowed features for a user
'''
    
def uid_to_features(user_id, user_apps, app_feats, user_feats):
    out_dict = {"user_id": user_id}
    # Obtaining the list of apps a user has
    user_app_ids = [app["app_id"] for app in user_apps]        
    # Extracting the features allowed for the user of interest
    user_features_allowed = get_target_array(user_id, "user_id", "features_allowed", user_feats) 
    
    app_permissions = []    
    for app_id in user_app_ids:
        app_report = {"app_id": app_id, "features_allowed": []}
        # Extracting the features available for an app
        app_features = get_target_array(app_id, "app_id", "features_available", app_feats)
        for app_feature in app_features:
            if app_feature in user_features_allowed:
                app_report["features_allowed"].append(app_feature)
        # Adding the complete app report for a particular app
        app_permissions.append(app_report)
                
    out_dict["application_permissions"] = app_permissions
                
    return out_dict


In [11]:
# Applying the function to the example scenario

uid = 1

uid_to_features(uid, user_apps, app_features, user_features)


{'user_id': 1,
 'application_permissions': [{'app_id': 1, 'features_allowed': [1, 2]},
  {'app_id': 2, 'features_allowed': [5]},
  {'app_id': 3, 'features_allowed': []},
  {'app_id': 126, 'features_allowed': []}]}

## Task 7

Write appropriate unit tests for your functions above

In [12]:
class Test_uid_to_features(unittest.TestCase):
    
    # Testing the outcome of the function for the given parameters
    def test_uid_to_features(self):
        self.assertEqual(uid_to_features(uid, user_apps, app_features, user_features), {'user_id': 1,
 'application_permissions': [{'app_id': 1, 'features_allowed': [1, 2]},
  {'app_id': 2, 'features_allowed': [5]},
  {'app_id': 3, 'features_allowed': []},
  {'app_id': 126, 'features_allowed': []}]})  

unittest.main(argv=[''], verbosity=2, exit=False)

test_getPlaybackTime (__main__.Test_getPlaybackTime) ... ok
test_getUsers (__main__.Test_getUsers) ... ok
test_uid_to_features (__main__.Test_uid_to_features) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.010s

OK


## Task 8

Identify any possible shortcomings or limitations of both your functions if any.

#### Task's 6 Limitations
1. "if line[lookup_key] == lookup_value:",- the assumption is that there are no missing dictionary keys in the data.
2. Doesn't scale well, in particular app_features where it loops through the each application that the user has.